In [1]:
import sys
sys.path.append('/usr/local/lib/python3.9/site-packages/')
import bisect
import math
import matplotlib as plt
import pandas as pd
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
import numpy as np
import copy
import json
import lxml
from lxml import objectify 

In [35]:
#dataframe mapping define mapping dict
#Dataframe defined by dicts.  indices are columns, array is rows in that column
foodframe = pd.DataFrame({"food":["Bacon","pastrami","turkey","chicken","pastrami"],"ounces":[1.1,2.3,5.4,2.2,2.6]},index=["a","b","c","d","e"])
print(foodframe)
#a map lets you create a column based on another column
foodtotype =    {
                "bacon":"yummy",
                "pastrami":"spicy",
                "turkey":"meh",
                "chicken":"meh"
                }
#convert to series then use map, then add to frame 
lowerfood = ((foodframe.food).str.lower())
foodframe["animal"]=lowerfood.map(foodtotype)
print(foodframe)

#replacement 
foodframe=foodframe.replace("pastrami","tacos")
print(foodframe)

#map can be used on indices 

print(foodframe.index)

indexmap =    {
                "a":"1",
                "b":"2",
                "c":"3",
                "d":"4",
                "e":"5"
                }

foodframe.index=foodframe.index.map(indexmap )
print(foodframe)
#can rename indices and colmns with "ddataframe.rename"




       food  ounces
a     Bacon     1.1
b  pastrami     2.3
c    turkey     5.4
d   chicken     2.2
e  pastrami     2.6
       food  ounces animal
a     Bacon     1.1  yummy
b  pastrami     2.3  spicy
c    turkey     5.4    meh
d   chicken     2.2    meh
e  pastrami     2.6  spicy
      food  ounces animal
a    Bacon     1.1  yummy
b    tacos     2.3  spicy
c   turkey     5.4    meh
d  chicken     2.2    meh
e    tacos     2.6  spicy
Index(['a', 'b', 'c', 'd', 'e'], dtype='object')
      food  ounces animal
1    Bacon     1.1  yummy
2    tacos     2.3  spicy
3   turkey     5.4    meh
4  chicken     2.2    meh
5    tacos     2.6  spicy


In [42]:
#pd allows for categorial data
age = [18,22,34,45,56,34,67,85,99]
bins = [16,18,27,55,100]
#pd.cut categorizes data
cats= pd.cut(age,bins)
print (cats)
#each index in cats gives the category of the data
#codes gives you the bin number
print (cats.codes)
#values_counts makes a histogram
print (pd.value_counts(cats))



[(16, 18], (18, 27], (27, 55], (27, 55], (55, 100], (27, 55], (55, 100], (55, 100], (55, 100]]
Categories (4, interval[int64]): [(16, 18] < (18, 27] < (27, 55] < (55, 100]]
[0 1 2 2 3 2 3 3 3]
(55, 100]    4
(27, 55]     3
(16, 18]     1
(18, 27]     1
dtype: int64


In [111]:
randoframe=pd.DataFrame(np.random.randn(1000,4))
#print(randoframe)
#describe prints out fun statistics
#print(randoframe.describe())

#filters
#yikes, iloc inverts axes...
#print(randoframe.iloc[2])
curser=randoframe[2]

#the "any" method selects all of an axis with this 
randoframe[(abs(randoframe)>3.).any(1)]


,0,1,2,3
31,-0.875377,3.093606,0.937485,1.181431
224,0.350991,-0.543942,-1.068517,3.257313
598,1.297219,3.993449,0.148692,0.177166
624,3.021194,0.520721,-0.295479,-0.383556
633,0.117978,1.104918,3.005476,0.251824
783,1.977321,3.335664,-0.635091,-0.796478
860,-0.510058,3.068192,0.483585,0.222226


In [122]:
# I think this is converting to a one-hot
sampledf=pd.DataFrame({"taco":["a","a","b","b","c","d","e","e","e"],"data":np.arange(9)})
pd.get_dummies(sampledf["taco"])

,a,b,c,d,e
0,1,0,0,0,0
1,1,0,0,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,0,1,0
6,0,0,0,0,1
7,0,0,0,0,1
8,0,0,0,0,1


In [130]:
#double indexing allows high dim representation in low dim
data=pd.Series(np.arange(9),index=[["a","a","a","b","b","b","c","c","c"],[1,3,4,2,3,4,2,1,3]])
print (data)
print (data["c"])
print (data[:,1])

a  1    0
   3    1
   4    2
b  2    3
   3    4
   4    5
c  2    6
   1    7
   3    8
dtype: int64
2    6
1    7
3    8
dtype: int64
a    0
c    7
dtype: int64


In [181]:
#unstack turns a double index Series into a DataFrame

dfdata=data.unstack()
print (dfdata)
print(dfdata.stack())


   zoop                                           gloop                  
      x                       y                       z                  
      1     2     3     4     1     2     3     4     1     2     3     4
a   0.0   NaN   3.0   6.0   1.0   NaN   4.0   7.0   2.0   NaN   5.0   8.0
b   NaN   9.0  12.0  15.0   NaN  10.0  13.0  16.0   NaN  11.0  14.0  17.0
c  21.0  18.0  24.0   NaN  22.0  19.0  25.0   NaN  23.0  20.0  26.0   NaN
    gloop  zoop      
        z     x     y
a 1   2.0   0.0   1.0
  3   5.0   3.0   4.0
  4   8.0   6.0   7.0
b 2  11.0   9.0  10.0
  3  14.0  12.0  13.0
  4  17.0  15.0  16.0
c 1  23.0  21.0  22.0
  2  20.0  18.0  19.0
  3  26.0  24.0  25.0


In [186]:
#columns dan be hierarchical as well
data=pd.DataFrame(np.arange(9*3).reshape(9,3),index=[["a","a","a","b","b","b","c","c","c"],[1,3,4,2,3,4,2,1,3]],columns=[["zoop","zoop","gloop"],["x","y","z"]])
print (data)
print (data.loc["a"])
print (data.loc["a"]["zoop"])
print (data.loc["a"]["zoop"]["x"])
print (data.loc["a"]["zoop"]["x"][4])
#Yikes


    zoop     gloop
       x   y     z
a 1    0   1     2
  3    3   4     5
  4    6   7     8
b 2    9  10    11
  3   12  13    14
  4   15  16    17
c 2   18  19    20
  1   21  22    23
  3   24  25    26
  zoop    gloop
     x  y     z
1    0  1     2
3    3  4     5
4    6  7     8
   x  y
1  0  1
3  3  4
4  6  7
1    0
3    3
4    6
Name: x, dtype: int64
6


In [192]:
print (data)
print (data.swaplevel(0,1,axis=0))
print (data.swaplevel(0,1,axis=1))

    zoop     gloop
       x   y     z
a 1    0   1     2
  3    3   4     5
  4    6   7     8
b 2    9  10    11
  3   12  13    14
  4   15  16    17
c 2   18  19    20
  1   21  22    23
  3   24  25    26
    zoop     gloop
       x   y     z
1 a    0   1     2
3 a    3   4     5
4 a    6   7     8
2 b    9  10    11
3 b   12  13    14
4 b   15  16    17
2 c   18  19    20
1 c   21  22    23
3 c   24  25    26
       x    y     z
    zoop zoop gloop
a 1    0    1     2
  3    3    4     5
  4    6    7     8
b 2    9   10    11
  3   12   13    14
  4   15   16    17
c 2   18   19    20
  1   21   22    23
  3   24   25    26


In [208]:
#operations on frame levels
print (data)
print (data.sum())

print (data.sum(level=0,axis=0))
print (data.sum(level=1,axis=0))
print (data.sum(level=0,axis=1))
print (data.sum(level=1,axis=1))



    zoop     gloop
       x   y     z
a 1    0   1     2
  3    3   4     5
  4    6   7     8
b 2    9  10    11
  3   12  13    14
  4   15  16    17
c 2   18  19    20
  1   21  22    23
  3   24  25    26
zoop   x    108
       y    117
gloop  z    126
dtype: int64
  zoop     gloop
     x   y     z
a    9  12    15
b   36  39    42
c   63  66    69
  zoop     gloop
     x   y     z
1   21  23    25
3   39  42    45
4   21  23    25
2   27  29    31
     zoop  gloop
a 1     1      2
  3     7      5
  4    13      8
b 2    19     11
  3    25     14
  4    31     17
c 2    37     20
  1    43     23
  3    49     26
      x   y   z
a 1   0   1   2
  3   3   4   5
  4   6   7   8
b 2   9  10  11
  3  12  13  14
  4  15  16  17
c 2  18  19  20
  1  21  22  23
  3  24  25  26


In [222]:
#combining datasets
#pandas.merge connects  rows
data=pd.DataFrame({"key":["a","b","b","c"],"data":np.arange(4)})
data1=pd.DataFrame({"key":["a","b","b","c","c","c","d"],"data1":np.arange(7)})
# I think this does all permutations?
print (data)
print (data1)
print(pd.merge(data1,data))
print(pd.merge(data1,data,on="key"))

  key  data
0   a     0
1   b     1
2   b     2
3   c     3
  key  data1
0   a      0
1   b      1
2   b      2
3   c      3
4   c      4
5   c      5
6   d      6
  key  data1  data
0   a      0     0
1   b      1     1
2   b      1     2
3   b      2     1
4   b      2     2
5   c      3     3
6   c      4     3
7   c      5     3
  key  data1  data
0   a      0     0
1   b      1     1
2   b      1     2
3   b      2     1
4   b      2     2
5   c      3     3
6   c      4     3
7   c      5     3


In [ ]:
# I have no idea, join exists too.  come back to